In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from scipy.stats import t, norm

In [ ]:
def scrap_table(url,headers):
  response = requests.get(url, headers=headers)
  if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    target_class = "ink-900"
    links = soup.find('a', class_=target_class)
    url2=url+links.get('href')
    response2 = requests.get(url2, headers=headers)
    soup2 = BeautifulSoup(response2.text, 'html.parser')
    data=[]
    table = soup.find('table')
    if table:
      rows = table.find_all('tr')
      headers = [th.text.strip() for th in rows[0].find_all('th')]
      for row in rows[1:]:
        cols = [td.text.strip() for td in row.find_all('td')]
        data.append(cols)

    table2 = soup2.find('table')
    if table2:
      rows = table2.find_all('tr')
      headers = [th.text.strip() for th in rows[0].find_all('th')]
      for row in rows[1:]:
        cols = [td.text.strip() for td in row.find_all('td')]
        data.append(cols)
    df = pd.DataFrame(data, columns=headers)
    return df

# **SCRAPING TICKERS**

In [ ]:
import requests
from bs4 import BeautifulSoup

def scrap_tickers(url,headers):
  response = requests.get(url, headers=headers)
  if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    target_class = "ink-900"
    links = soup.find('a', class_=target_class)
    target_value = "_blank"
    links = soup.find_all('a', attrs={'target': target_value})
    href_links = [link['href'] for link in links if 'href' in link.attrs]
    ticker=[]
    for href in href_links:
      parts = href.split('/')
      result = parts[2] if len(parts) > 2 else None
      ticker.append(result+".NS")
      return ticker


In [ ]:
url = "https://www.screener.in/company/NIFTY/"
url2 = "https://www.screener.in/company/NIFTY/?page=2"
headers = {'User-Agent': 'Mozilla/5.0'}
data=scrap_table(url,headers)
data2=scrap_table(url2,headers)
data=pd.concat([data,data2],ignore_index=True)
data
data_final=pd.DataFrame()
data_final=data[['S.No.','Name','CMP\n                    Rs.','P/E','Mar Cap\n                    Rs.Cr.']].copy()
data_final.rename(columns={'CMP\n                    Rs.': 'LTP','Mar Cap\n                    Rs.Cr.':'Market Cap'}, inplace=True)
data_final=data_final.dropna()
data_final=data_final.reset_index()
temp=0
for i in range(len(data_final)-1):
  if data_final['Name'].iloc[i]=="Median: 50 Co.":
    if(temp==0):
      data_final=data_final.drop([i])
      temp=1
    else:
      data_final=data_final.drop([i+1])
data_final=data_final.reset_index()
data_final=data_final.drop(['index','level_0'],axis=1)
ticker_list=scrap_tickers(url,headers)
ticker_list2=scrap_tickers(url2,headers)
ticker_list.extend(ticker_list2)
ticker_list
data_final

Extracted Links:


,S.No.,Name,LTP,P/E,Market Cap,52_week_high,52_week_low
0,1.,Reliance Industr,1272.85,25.36,1722467.61,"1,609","1,217"
1,2.,TCS,4473.90,33.61,1618696.17,"4,592","3,592"
2,3.,HDFC Bank,1871.75,20.68,1430928.71,"1,880","1,363"
3,4.,Bharti Airtel,1681.75,71.04,1007452.46,"1,779",960
4,5.,ICICI Bank,1344.90,20.04,949165.22,"1,362",970
5,6.,Infosys,1999.70,30.81,830324.82,"2,007","1,358"
6,7.,St Bk of India,861.55,10.04,768900.64,912,601
7,8.,ITC,470.00,28.61,588008.16,529,399
8,9.,Hind. Unilever,2390.10,54.64,561575.81,"3,035","2,170"
9,10.,Larsen & Toubro,3887.00,39.70,534513.28,"3,964","3,175"


In [ ]:
def ohlc(ticker,start,end):
  data=yf.download(ticker,start,end)
  perc= ((data.iloc[len(data)-1]['Close']-data.iloc[0]['Close'])*100)/data.iloc[0]['Close']
  return perc.values[0]

def ohlcv(ticker,start,end):
  data=yf.download(ticker,start,end)
  return data.iloc[len(data)-1]['Volume'].values[0]
return_6mopercent=[]
return_1yrpercent=[]
return_5yrpercent=[]
return_volume=[]
try:
  for i in range(len(ticker_list)):
    return_6mopercent.append(ohlc(ticker_list[i],'2024-06-14','2024-12-14'))
    return_1yrpercent.append(ohlc(ticker_list[i],'2023-12-14','2024-12-14'))
    return_5yrpercent.append(ohlc(ticker_list[i],'2019-12-14','2024-12-14'))
except Exception as e:
  print("k")
for i in range(len(ticker_list)):
  return_volume.append(ohlcv(ticker_list[i],'2024-11-14','2024-12-14'))
data_final['6Months returns in%']=return_6mopercent
data_final['1Year returns in%']=return_1yrpercent
data_final['5Years returns in%']=return_5yrpercent
data_final['Volume']=return_volume
data_final

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

,S.No.,Name,LTP,P/E,Market Cap,52_week_high,52_week_low,6Months returns in%,1Year returns in%,5Years returns in%,Volume
0,1.,Reliance Industr,1272.85,25.36,1722467.61,"1,609","1,217",-13.854020,3.309460,77.723484,28630222.0
1,2.,TCS,4473.90,33.61,1618696.17,"4,592","3,592",16.749516,21.996043,110.363226,1967048.0
2,3.,HDFC Bank,1871.75,20.68,1430928.71,"1,880","1,363",17.211470,13.429081,48.864679,9504501.0
3,4.,Bharti Airtel,1681.75,71.04,1007452.46,"1,779",960,17.819110,67.305012,306.145529,12900707.0
4,5.,ICICI Bank,1344.90,20.04,949165.22,"1,362",970,21.638854,30.111739,149.401952,13536684.0
5,6.,Infosys,1999.70,30.81,830324.82,"2,007","1,358",34.307201,33.184589,180.070021,5362693.0
6,7.,St Bk of India,861.55,10.04,768900.64,912,601,2.663248,38.146389,159.620302,9379370.0
7,8.,ITC,470.00,28.61,588008.16,529,399,9.010787,2.151705,98.395953,31753858.0
8,9.,Hind. Unilever,2390.10,54.64,561575.81,"3,035","2,170",-3.615280,-5.017187,21.033047,2260902.0
9,10.,Larsen & Toubro,3887.00,39.70,534513.28,"3,964","3,175",5.401593,13.221284,198.311589,2300361.0
